<a href="https://colab.research.google.com/github/vnylp/JupyterNotebookCodes/blob/master/Green_Street_Advisors_Python_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Background:
• We have collected data on single-family homes that we want to analyze.
• [data_property.csv] contains a list of properties, but we are unsure of how "clean" the data is.
• [data_rent.csv] contains rent data for these properties that was collected throughout 2019 (assume that this data has already been cleaned).
• While analyzing the data, if you make any assumptions, explain what they are in comments.
• When you are finished, submit the code that you have written and your output csv files, zipped into a compressed folder.

1. Write code to parse / ingest [data_property.csv].
2. Explore the data set, looking for potential errors, and clean the data set (this can be done manually by editing data_property.csv or programatically).
3. Explain your methodology from #2 (this can be done in comments in your code).
4. For each state: calculate the minimum square footage, maximum square footage, and the average age in years of all properties (age = current year - year_built).
5. Output the results to a csv file, matching the [output_example_1.csv] format.

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_property=pd.read_csv('/content/data_property.csv')

In [10]:
from pandas_profiling import ProfileReport

Checking the null values

In [4]:
data_property.isna().sum()

property_id     0
address         0
city            0
state           0
zip             0
baths           0
beds            0
sf              0
year_built     29
dtype: int64

checking the data type of null values : Catgoerical or Intger

In [6]:
data_property.dtypes

property_id      int64
address         object
city            object
state           object
zip             object
baths          float64
beds             int64
sf              object
year_built      object
dtype: object

In [15]:
data_property.shape[0]

766

Percentage of missing value is less than 5%

In [17]:
29/766*100

3.7859007832898173

Still if we have to fill null data , simple approach is to look at mean median mode values. For categorical value we can use Mode more commonly.

In [58]:
data_property['year_built']=data_property['year_built'].fillna(data_property['year_built'].mode()[0])

In [62]:
data_property['year_built'].value_counts().index

Index(['2004', '2005', '2006', '2003', '2002', '1999', '2007', '2001', '2000',
       '1997', '1998', '1996', '2008', '1995', '1994', '1989', '2009', '1990',
       '1986', '2013', '1987', '1971', '1980', '1988', '1993', '1954', '1952',
       '1925', '1985', '1974', '1992', '1951', '1977', '1955', '1970', '1950',
       '1957', '1968', '1956', '1983', '2012', '1964', '1960', '1978', '1924',
       '1917', '1984', '1973', '1962', '1981', '1959', '2010', '1947', '1926',
       '1991', '1953', '1963', '2011', '1972', '2030', 'x', '1930', '1949',
       '1975', '1938', '1907', '1935', '20009', '1958', '1976', '680', '1961',
       '1942', '1929', '0', '1939', '1966', '203'],
      dtype='object')

After looking at a set of years , data added seem to be wrong, so to correct need to identify wrong periods.

In [63]:
wrongyears=['2030', 'x',  '20009', '680', '0', '203']

We will correct these rows with imputing mode value

In [67]:
data_property.loc[data_property['year_built'].isin(wrongyears),'year_built']=data_property['year_built'].mode()[0]

In [68]:
data_property['year_built'].value_counts().index

Index(['2004', '2005', '2006', '2003', '2002', '1999', '2007', '2001', '2000',
       '1997', '1998', '1996', '2008', '1995', '1989', '1994', '2009', '1986',
       '1990', '2013', '1987', '1988', '1980', '1971', '1952', '1993', '1954',
       '1951', '1974', '1992', '1985', '1925', '1950', '1977', '2012', '1964',
       '1957', '1983', '1970', '1956', '1960', '1968', '1955', '2010', '1924',
       '1991', '1984', '1973', '1926', '1959', '1962', '1978', '1917', '1953',
       '1981', '1947', '1975', '1972', '1958', '1961', '1942', '1929', '1938',
       '1939', '1930', '2011', '1907', '1966', '1949', '1963', '1935', '1976'],
      dtype='object')

In [70]:
data_property['age']=2022 - data_property['year_built'].astype(int)

Years clean up is completed

In [71]:
sf_min=data_property[['state','sf']].groupby('state').min().reset_index()
sf_max=data_property[['state','sf']].groupby('state').max().reset_index()
age_avg=data_property[['state','age']].groupby('state').mean().reset_index()

In [76]:
df= sf_min.merge(sf_max,on='state').merge(age_avg,on='state')

In [78]:
df.columns=['state', 'sf_min', 'sf_max', 'age_avg']

In [80]:
df.to_csv('output_example_1.csv')

Problem statement 2


6. Parse / ingest [data_rent.csv].
7. Calculate how many rent data points were collected for each property and output to a csv file, matching the [output_example_2.csv] format.
8. Calculate how many properties had a rent increase during 2019 and output to the console.
9. Join the 2 data sets to calculate how many CA properties have a rent observation on 2019-05-27 and output to console.
10. Given a larger data set and more time, explain how your cleaning methodology from step 2 would change (this can be done in comments in your code).

In [81]:
data_rent=pd.read_csv('/content/data_rent.csv')

Calculate how many rent data points were collected for each property

In [86]:
rentdatapoints=data_rent['property_id'].value_counts().reset_index().sort_values('index')
rentdatapoints.columns=['property_id','number_observations_rent']

In [88]:
rentdatapoints.to_csv('output_example_2.csv')

Join the 2 data sets to calculate how many CA properties have a rent observation on 2019-05-27 and output to console.

In [96]:
data_rentupdated=data_rent.merge(data_property[['property_id','state']],on='property_id',how='left')

In [99]:
rentobserved= data_rentupdated[(data_rentupdated['state']=='CA') & (data_rentupdated['date'].astype(str)=='5/27/2019')]

Given a larger data set and more time, explain how your cleaning methodology from step 2 would change (this can be done in comments in your code)

1. Some of the very simple techniques we can use is we can use ML approach to impute the missing values.
2. But even before jumping on to ML we will keep it simple using mean , median and mode values. We will test our model accuracy with that.
3. We can try KNN algorithm to fill null values or we can try decision trees for this particular use case.
5. There is a SMOTE technique as well to fill null values.
6. With each trial and error we will keep on testing model accuracy to understand relevant approach to pic and choose